In [1]:
!pip install -U spacy

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [2]:
# The google-cloud-vision package already installed in colab, no need to install.
!pip install --upgrade google-cloud-vision

     |████████████████████████████████| 440kB 9.0MB/s 


In [3]:
# Install pdf2image library
!pip install pdf2image

In [4]:
# Install poppler required for pdf2image
!apt-get install poppler-utils 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 31 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.10 [154 kB]
Fetched 154 kB in 0s (1,537 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.10_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.10) ...
Setting up poppler-utils (0.62.0-2ubuntu2.10) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
import os
from google.cloud import vision
import io
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Extract text from images.

In [0]:
def detect_text_from_image(path):
    '''
    Function to extract texts from a image using GOOGLE CLOUD VISION API.
    Param:
        path : Full path of image.
    Return:
        Extracted text.
    '''
    # 
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/My First Project-dff60b5d1137.json"
    """Detects text in the file."""
    
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.document_text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

    return texts[0].description

# Detect text from PDF.

In [17]:
detect_text_from_image('/content/form_ocr_ner/Merrilllynch_CRA/RobertNSanford.png')

FileNotFoundError: ignored

# Extract text from PDF

In [0]:
from pdf2image import convert_from_path
from pathlib import Path

In [0]:
def detect_text_from_pdf(pdf_path, num_of_pages):

    # Extract customer_name from file name. Replace ' ' by '_'
    customer_name = Path(pdf_path).stem.replace(' ', '_')

    pages = convert_from_path(pdf_path, 100, fmt = 'jpg')

    pdf_text = []
    for i, page in enumerate(pages[0:num_of_pages]):
        temp_img_path = customer_name+ str(i) + '.jpg'
        page.save(temp_img_path)
        temp_text = detect_text_from_image(temp_img_path)
        pdf_text.append(temp_text)
        os.remove(temp_img_path)
    return '. '.join(pdf_text).replace("\n", " ")

# Generate list of files

In [0]:
from pathlib import PurePath

In [0]:
root_paths = ['/content/drive/My Drive/DOC-NER/Ameritrade Institutional',
              '/content/drive/My Drive/DOC-NER/Merrilllynch_CRA',
              '/content/drive/My Drive/DOC-NER/Schwab_IRA']

all_files_list = []
for root_path in root_paths:
    for path, subdirs, files in os.walk(root_path):
        for name in files:
            all_files_list.append(os.path.join(path, name))

In [22]:
all_files_list

['/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Zita J Rottman_.pdf',
 '/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Teresa R Woods_.pdf',
 '/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Susan T Green_.pdf',
 '/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Robert R Sheppard_.pdf',
 '/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Robert R Allan_.pdf',
 '/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Robert J Barr_.pdf',
 '/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Pamela E Byrd_.pdf',
 '/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Nora R Mills_.pdf',
 '/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Michael_C_Preston_.pdf',
 '/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Mary H Craven_.pdf',
 '/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Mary H Craven_ (1).pdf',
 '/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Marie P Watkins_.pdf',
 '/content/drive/My Driv

In [23]:
from tqdm import tqdm
extracted_text = []
customer_file_name = []

for path_ in tqdm(all_files_list):
    print(path_)
    file_name = path_.split('/')[-1]
    print(file_name)
    
    if Path(file_name).suffix == '.pdf':
        extracted_text.append(detect_text_from_pdf(pdf_path = path_, num_of_pages = 2))
        customer_file_name.append(file_name)
    elif Path(file_name).suffix == '.png' or 'jpg':
        extracted_text.append(detect_text_from_image(path_))
        customer_file_name.append(file_name)
    else:
        pass

  0%|          | 0/72 [00:00<?, ?it/s]

/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Zita J Rottman_.pdf
Zita J Rottman_.pdf
Texts:


  1%|▏         | 1/72 [00:01<01:38,  1.39s/it]

Texts:
/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Teresa R Woods_.pdf
Teresa R Woods_.pdf
Texts:


  3%|▎         | 2/72 [00:03<01:47,  1.53s/it]

...

Texts:
/content/drive/My Drive/DOC-NER/Schwab_IRA/Karen_A_Sliva_.pdf
Karen_A_Sliva_.pdf
Texts:


 99%|█████████▊| 71/72 [02:16<00:02,  2.42s/it]

Texts:
/content/drive/My Drive/DOC-NER/Schwab_IRA/Patricia_J_Kim_.pdf
Patricia_J_Kim_.pdf
Texts:


100%|██████████| 72/72 [02:19<00:00,  1.94s/it]

Texts:


In [25]:
print(extracted_text[0])

D Ameritrade Institutional Reset Form ACCOUNT TRANSFER FORM TO TRANSFER YOUR EXISTING ACCOUNT TO TD AMERITRADE Account # 896547896 Advisor Code 987456932 Case # 1 PLEASE ATTACH A COPY OF YOUR LATEST STATEMENT. TD AMERITRADE ACCOUNT INFORMATION Account Title/Registration: Zita J Rottman Social Security Number/Tax ID: Clearing Number 404711495 0188 Individual Personal Trust Limited Liability Company Roth IRA Qualified Retirement Plan (Non-Qualified) Employer Plan Type: Joint Estate Partnership/Investment Club SEP IRA Simple IRA UTMA/UGMA Corporate Traditional IRA/Rollover RA Coverdell ESA Beneficiary IRA Other: 2 INFORMATION ON THE ACCOUNT YOU ARE TRANSFERRING FROM Account Title/Registration as shown on your statement Account Number Zita JRottman 870214590 Name of Delivering Firm: NVDA Physical Address of Firm (no PO BOX): Telephone Number (required): 200 Crescent Ct #1200 (541) 754-3010 Individual Personal Trust Limited Liability Company Roth IRA Qualified Retirement Plan (Non-Qualified

In [0]:
master_df = pd.DataFrame({'file_name' : customer_file_name,
                          'extracted_text' : extracted_text})

In [27]:
master_df.head()

,file_name,extracted_text
0,Zita J Rottman_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...
1,Teresa R Woods_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...
2,Susan T Green_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...
3,Robert R Sheppard_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...
4,Robert R Allan_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...


In [0]:
#master_df.to_csv('master_data.csv', encoding='utf-8', index=False)

In [0]:
master_df['FILE NAME'] = master_df['file_name'].apply(lambda x: Path(x).stem)

In [30]:
master_df.head()

,file_name,extracted_text,FILE NAME
0,Zita J Rottman_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Zita J Rottman_
1,Teresa R Woods_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Teresa R Woods_
2,Susan T Green_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Susan T Green_
3,Robert R Sheppard_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Robert R Sheppard_
4,Robert R Allan_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Robert R Allan_


In [0]:
ira_df = pd.read_csv('/content/drive/My Drive/DOC-NER/IRA.csv')

In [33]:
ira_df.head()

,SLNo.,CATEGORY,FILE NAME,FILE SIZE,TITLE,DESCRIPTION,CLIENT,ADDRESS,SOCIAL SECURITY NO,RELATIONSHIP NO.,ACC NO,ACC TYPE,PHONE,CITY,EMAIL
0,0,Merrilllynch_CRA,Adrian Clemens,298 KB,Client Account Transfer Form,Use this form to transfer all or a portion of ...,Daniel C Paul,2 Seaport Ln,545401875,335-0012,89101454,IRA,5417542545,"Boston, MA,02210",Adrian@gmail.com
1,1,Merrilllynch_CRA,Peter E Trosper,295 KB,Client Account Transfer Form,Use this form to transfer all or a portion of ...,Grace P Lane,450 Park Ave #18,593746465,335-8978,66667894,IRA,5417543022,"New York, NY,10022",pater@gmail.com
2,2,Merrilllynch_CRA,Richard K Kral,298 KB,Client Account Transfer Form,Use this form to transfer all or a portion of ...,Jeanette B Hughes,340 Madison Ave,344466775,335-2125,79703212,IRA,5417543888,"New York, NY,10173",Richard@gmail.com
3,3,Merrilllynch_CRA,Robert E Regan,298 KB,Client Account Transfer Form,Use this form to transfer all or a portion of ...,Grace M Gracia,Search Results 11988 El Camino Real #600,417414892,335-0021,NaN,IRA,5417543010,"San Diego, CA,92130",Robert@gmail.com
4,4,Merrilllynch_CRA,Rose R Webb,298 KB,Client Account Transfer Form,Use this form to transfer all or a portion of ...,Donald S Cordova,11150CA-2 #200,539236318,335-4587,22560564,IRA,5417545879,"Los Angeles, CA,90025",Rose@gmail.com


In [0]:
final_df = pd.merge(master_df, ira_df, on='FILE NAME')

In [0]:
final_df.columns = ['file_name', 'extracted_text', 'FILE_NAME', 'SLNo', 'CATEGORY',
       'FILE_SIZE', 'TITLE', 'DESCRIPTION', 'CLIENT', 'ADDRESS',
       'SOCIAL_SECURITY_NO', 'RELATIONSHIP_NO', 'ACC_NO', 'ACC_TYPE', 'PHONE',
       'CITY', 'EMAIL']

In [0]:
#final_df.to_csv('final_df.csv', encoding='utf-8', index=False)

In [0]:
final_df['PHONE'] = final_df['PHONE'].apply(lambda x: '('+str(x)[:3]+')'+' '+str(x)[3:6]+'-'+str(x)[6:10])

In [0]:
for i in final_df.columns:
    final_df[i] = final_df[i].astype('str') 

In [0]:
final_df.to_csv('final_df.csv', encoding='utf-8', index=False)

In [40]:
final_df.dtypes

file_name             object
extracted_text        object
FILE_NAME             object
SLNo                  object
CATEGORY              object
FILE_SIZE             object
TITLE                 object
DESCRIPTION           object
CLIENT                object
ADDRESS               object
SOCIAL_SECURITY_NO    object
RELATIONSHIP_NO       object
ACC_NO                object
ACC_TYPE              object
PHONE                 object
CITY                  object
EMAIL                 object
dtype: object

# NER

In [0]:
import itertools

In [0]:
final_df = pd.read_csv("final_df.csv")
for i in final_df.columns:
    final_df[i] = final_df[i].astype('str') 

In [0]:
final_df.columns = ['file_name', 'extracted_text', 'FILE_NAME', 'SLNo', 'CATEGORY',
       'FILE_SIZE', 'TITLE', 'DESCRIPTION', 'CLIENT', 'ADDRESS',
       'SOCIAL_SECURITY_NO', 'RELATIONSHIP_NO', 'ACC_NO', 'ACC_TYPE', 'PHONE',
       'CITY', 'EMAIL']

In [0]:
final_df['TITLE'] = final_df['TITLE'].apply(lambda x: x.replace('IRA ', ''))

In [45]:
final_df

,file_name,extracted_text,FILE_NAME,SLNo,CATEGORY,FILE_SIZE,TITLE,DESCRIPTION,CLIENT,ADDRESS,SOCIAL_SECURITY_NO,RELATIONSHIP_NO,ACC_NO,ACC_TYPE,PHONE,CITY,EMAIL
0,Zita J Rottman_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Zita J Rottman_,40,Ameritrade Institutional,142 KB,ACCOUNT TRANSFER FORM,A. Transfer from an ACAT eligible Brokerage Fi...,Zita J Rottman,200 Crescent Ct#1200,404711495,nan,870214590,NVDA,(541) 754-3010,nan,nan
1,Teresa R Woods_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Teresa R Woods_,59,Ameritrade Institutional,142 KB,ACCOUNT TRANSFER FORM,A. Transfer from an ACAT eligible Brokerage Fi...,Teresa R Woods,3403 Richards Avenue,139394344,nan,875032122,BABA,(541) 754-1100,nan,nan
2,Susan T Green_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Susan T Green_,28,Ameritrade Institutional,142 KB,ACCOUNT TRANSFER FORM,A. Transfer from an ACAT eligible Brokerage Fi...,Susan T Green,3116 Wright Court,517144655,nan,856901266,TGT,(541) 754-3044,nan,nan
3,Robert R Sheppard_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Robert R Sheppard_,27,Ameritrade Institutional,142 KB,ACCOUNT TRANSFER FORM,A. Transfer from an ACAT eligible Brokerage Fi...,Robert R Sheppard,3291 Hall Street,545401844,nan,987012777,NVDA,(541) 754-3222,nan,nan
4,Robert J Barr_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Robert J Barr_,26,Ameritrade Institutional,142 KB,ACCOUNT TRANSFER FORM,A. Transfer from an ACAT eligible Brokerage Fi...,Robert J Barr,4567 Willison Street,139394322,nan,875032188,BABA,(541) 754-3022,nan,nan
5,Pamela E Byrd_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Pamela E Byrd_,25,Ameritrade Institutional,142 KB,ACCOUNT TRANSFER FORM,A. Transfer from an ACAT eligible Brokerage Fi...,Pamela E Byrd,514 Stutler Lane,139394344,nan,875032122,GOOG,(541) 754-3033,nan,nan
6,Nora R Mills_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Nora R Mills_,57,Ameritrade Institutional,142 KB,ACCOUNT TRANSFER FORM,A. Transfer from an ACAT eligible Brokerage Fi...,Nora R Mills,4010 Mulberry Street,545401870,nan,987012344,JNJ,(541) 754-3055,nan,nan
7,Michael_C_Preston_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Michael_C_Preston_,48,Ameritrade Institutional,142 KB,ACCOUNT TRANSFER FORM,A. Transfer from an ACAT eligible Brokerage Fi...,Michael C Preston,4645 Hope Street,139394339,nan,875032106,BBY,(541) 754-3044,nan,nan
8,Mary H Craven_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Mary H Craven_,24,Ameritrade Institutional,142 KB,ACCOUNT TRANSFER FORM,A. Transfer from an ACAT eligible Brokerage Fi...,Mary H Craven,2397 Deans Lane,545401875,nan,987012322,JNJ,(541) 754-3055,nan,nan
9,Marie P Watkins_.pdf,D Ameritrade Institutional Reset Form ACCOUNT ...,Marie P Watkins_,38,Ameritrade Institutional,142 KB,ACCOUNT TRANSFER FORM,A. Transfer from an ACAT eligible Brokerage Fi...,Marie P Watkins,222 Berkeley St #15,517144661,nan,856901203,TGT,(541) 754-3012,nan,nan


In [0]:
from sklearn.utils import shuffle

In [0]:
from sklearn.model_selection import train_test_split
import spacy
from spacy.util import decaying
from spacy.matcher import PhraseMatcher
from spacy.lang.en import English
from spacy.util import minibatch, compounding
from spacy.gold import GoldParse 
from spacy.scorer import Scorer
import random

In [0]:
final_df = final_df.sample(frac=1)
final_df.reset_index(drop = True, inplace = True)

train, test = train_test_split(final_df, test_size=0.1)

In [49]:
train.shape, test.shape

((54, 17), (6, 17))

In [50]:
['TITLE', 'DESCRIPTION', 'CLIENT', 'ADDRESS',
       'SOCIAL_SECURITY_NO', 'RELATIONSHIP_NO', 'ACC_NO', 'ACC_TYPE', 'PHONE',
       'CITY', 'EMAIL']

['TITLE',
 'DESCRIPTION',
 'CLIENT',
 'ADDRESS',
 'SOCIAL_SECURITY_NO',
 'RELATIONSHIP_NO',
 'ACC_NO',
 'ACC_TYPE',
 'PHONE',
 'CITY',
 'EMAIL']

In [0]:
nlp = English()
#nlp = spacy.load("en_trf_bertbaseuncased_lg")
#nlp.vocab, attr='lemma', validate=True

In [0]:
#TITLE_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
TITLE_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
TITLE_patterns = [nlp(i) for i in final_df['TITLE'].to_list()]
TITLE_matcher.add("TITLE", None,*TITLE_patterns)

In [0]:
DESCRIPTION_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
DESCRIPTION_patterns = [nlp(i) for i in final_df['DESCRIPTION'].to_list()]
DESCRIPTION_matcher.add("DESCRIPTION", None,*DESCRIPTION_patterns)

In [0]:
CLIENT_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
CLIENT_patterns = [nlp(i) for i in final_df['CLIENT'].to_list()]
CLIENT_matcher.add("CLIENT", None,*CLIENT_patterns)

In [0]:

ADDRESS_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
ADDRESS_patterns = [nlp(i) for i in final_df['ADDRESS'].to_list()]
ADDRESS_matcher.add("ADDRESS", None,*ADDRESS_patterns)

In [0]:
SOCIAL_SECURITY_NO_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
SOCIAL_SECURITY_NO_patterns = [nlp(i) for i in final_df['SOCIAL_SECURITY_NO'].to_list()]
SOCIAL_SECURITY_NO_matcher.add("SOCIAL_SECURITY_NO", None,*SOCIAL_SECURITY_NO_patterns)

In [0]:
RELATIONSHIP_NO_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
RELATIONSHIP_NO_patterns = [nlp(i) for i in final_df['RELATIONSHIP_NO'].to_list()]
RELATIONSHIP_NO_matcher.add("RELATIONSHIP_NO", None,*RELATIONSHIP_NO_patterns)

In [0]:
ACC_NO_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
ACC_NO_patterns = [nlp(i) for i in final_df['ACC_NO'].to_list()]
ACC_NO_matcher.add("ACC_NO", None,*ACC_NO_patterns)

In [0]:
ACC_TYPE_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
ACC_TYPE_patterns = [nlp(i) for i in final_df['ACC_TYPE'].to_list()]
ACC_TYPE_matcher.add("ACC_TYPE", None,*ACC_TYPE_patterns)

In [0]:
PHONE_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
PHONE_patterns = [nlp(i) for i in final_df['PHONE'].to_list()]
PHONE_matcher.add("PHONE", None,*PHONE_patterns)

In [0]:
CITY_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
CITY_patterns = [nlp(i) for i in final_df['CITY'].to_list()]
CITY_matcher.add("CITY", None,*CITY_patterns)

In [0]:
EMAIL_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
EMAIL_patterns = [nlp(i) for i in final_df['EMAIL'].to_list()]
EMAIL_matcher.add("EMAIL", None,*EMAIL_patterns)

In [0]:
TRAIN_TEXTS = train['extracted_text'].tolist()
TEST_TEXTS = test['extracted_text'].tolist()

In [65]:
TRAINING_DATA = []

# Create a Doc object for each text in TEXTS
for doc in tqdm(nlp.pipe(TRAIN_TEXTS)):
    # Match on the doc and create a list of matched spans
    TITLE_spans = [doc[start:end] for match_id, start, end in TITLE_matcher(doc)]
    DESCRIPTION_spans = [doc[start:end] for match_id, start, end in DESCRIPTION_matcher(doc)]
    CLIENT_spans = [doc[start:end] for match_id, start, end in CLIENT_matcher(doc)]
    ADDRESS_spans = [doc[start:end] for match_id, start, end in ADDRESS_matcher(doc)]
    SOCIAL_SECURITY_NO_spans = [doc[start:end] for match_id, start, end in SOCIAL_SECURITY_NO_matcher(doc)]
    RELATIONSHIP_NO_spans = [doc[start:end] for match_id, start, end in RELATIONSHIP_NO_matcher(doc)]
    ACC_NO_spans = [doc[start:end] for match_id, start, end in ACC_NO_matcher(doc)]
    ACC_TYPE_spans = [doc[start:end] for match_id, start, end in ACC_TYPE_matcher(doc)]
    PHONE_spans = [doc[start:end] for match_id, start, end in PHONE_matcher(doc)]
    CITY_spans = [doc[start:end] for match_id, start, end in CITY_matcher(doc)]
    EMAIL_spans = [doc[start:end] for match_id, start, end in EMAIL_matcher(doc)]
    
    # Get (start character, end character, label) tuples of matches
    entities = [(span.start_char, span.end_char, "TITLE") for span in TITLE_spans]
    entities.extend([(span.start_char, span.end_char, "DESCRIPTION") for span in DESCRIPTION_spans])
    entities.extend([(span.start_char, span.end_char, "CLIENT") for span in CLIENT_spans])
    entities.extend([(span.start_char, span.end_char, "ADDRESS") for span in ADDRESS_spans])
    entities.extend([(span.start_char, span.end_char, "SOCIAL_SECURITY_NO") for span in SOCIAL_SECURITY_NO_spans])
    entities.extend([(span.start_char, span.end_char, "RELATIONSHIP_NO") for span in RELATIONSHIP_NO_spans])
    entities.extend([(span.start_char, span.end_char, "ACC_NO") for span in ACC_NO_spans])
    entities.extend([(span.start_char, span.end_char, "ACC_TYPE") for span in ACC_TYPE_spans])
    entities.extend([(span.start_char, span.end_char, "PHONE") for span in PHONE_spans])
    entities.extend([(span.start_char, span.end_char, "CITY") for span in CITY_spans])
    entities.extend([(span.start_char, span.end_char, "EMAIL") for span in EMAIL_spans])

    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    # Append the example to the training data
    TRAINING_DATA.append(training_example)

54it [00:00, 471.72it/s]


In [0]:
# Remove repetated of same type entity
for i,_ in enumerate(TRAINING_DATA):
    TRAINING_DATA[i][1]['entities'] = ([next(b) for a, b in itertools.groupby( 
                         TRAINING_DATA[i][1]['entities'], lambda y: y[2])]) 

In [67]:
TRAINING_DATA[0]

('D Ameritrade Institutional Reset Form ACCOUNT TRANSFER FORM TO TRANSFER YOUR EXISTING ACCOUNT TO TD AMERITRADE Account # 125014569 Advisor Code 333015558 Case # 1 PLEASE ATTACH A COPY OF YOUR LATEST STATEMENT. TD AMERITRADE ACCOUNT INFORMATION Account Title/Registration: Adrian T Clemens Social Security Number Tax ID: Clearing Number 545401875 0188 Personal Trust Limited Liability Company Roth IRA Individual (Non-Qualified) Joint UTMA/UGMA Qualified Retirement Plan Employer Plan Type: Simple IRA Estate SEP IRA Partnership/Investment Club Traditional IRA/Rollover RA Corporate Coverdell ESA Beneficiary IRA Other: 2 INFORMATION ON THE ACCOUNT YOU ARE TRANSFERRING FROM Account Title Registration as shown on your statement: Account Number: Adrian T Clemens 987012301 Name of Delivering Firm: JNJ Physical Address of Firm (no PO BOX): Telephone Number (required): 280 Park Ave 12th Floor (541) 754-3011 Individual Personal Trust Limited Liability Company Roth IRA Qualified Retirement Plan (Non

In [68]:
TESTING_DATA = []

# Create a Doc object for each text in TEXTS
for doc in tqdm(nlp.pipe(TEST_TEXTS)):
    # Match on the doc and create a list of matched spans
    TITLE_spans = [doc[start:end] for match_id, start, end in TITLE_matcher(doc)]
    DESCRIPTION_spans = [doc[start:end] for match_id, start, end in DESCRIPTION_matcher(doc)]
    CLIENT_spans = [doc[start:end] for match_id, start, end in CLIENT_matcher(doc)]
    ADDRESS_spans = [doc[start:end] for match_id, start, end in ADDRESS_matcher(doc)]
    SOCIAL_SECURITY_NO_spans = [doc[start:end] for match_id, start, end in SOCIAL_SECURITY_NO_matcher(doc)]
    RELATIONSHIP_NO_spans = [doc[start:end] for match_id, start, end in RELATIONSHIP_NO_matcher(doc)]
    ACC_NO_spans = [doc[start:end] for match_id, start, end in ACC_NO_matcher(doc)]
    ACC_TYPE_spans = [doc[start:end] for match_id, start, end in ACC_TYPE_matcher(doc)]
    PHONE_spans = [doc[start:end] for match_id, start, end in PHONE_matcher(doc)]
    CITY_spans = [doc[start:end] for match_id, start, end in CITY_matcher(doc)]
    EMAIL_spans = [doc[start:end] for match_id, start, end in EMAIL_matcher(doc)]
    
    # Get (start character, end character, label) tuples of matches
    entities = [(span.start_char, span.end_char, "TITLE") for span in TITLE_spans]
    entities.extend([(span.start_char, span.end_char, "DESCRIPTION") for span in DESCRIPTION_spans])
    entities.extend([(span.start_char, span.end_char, "CLIENT") for span in CLIENT_spans])
    entities.extend([(span.start_char, span.end_char, "ADDRESS") for span in ADDRESS_spans])
    entities.extend([(span.start_char, span.end_char, "SOCIAL_SECURITY_NO") for span in SOCIAL_SECURITY_NO_spans])
    entities.extend([(span.start_char, span.end_char, "RELATIONSHIP_NO") for span in RELATIONSHIP_NO_spans])
    entities.extend([(span.start_char, span.end_char, "ACC_NO") for span in ACC_NO_spans])
    entities.extend([(span.start_char, span.end_char, "ACC_TYPE") for span in ACC_TYPE_spans])
    entities.extend([(span.start_char, span.end_char, "PHONE") for span in PHONE_spans])
    entities.extend([(span.start_char, span.end_char, "CITY") for span in CITY_spans])
    entities.extend([(span.start_char, span.end_char, "EMAIL") for span in EMAIL_spans])

    # Format the matches as a (doc.text, entities) tuple
    testing_example = (doc.text, {"entities": entities})
    # Append the example to the testing data
    TESTING_DATA.append(testing_example)

6it [00:00, 501.72it/s]


In [0]:
# Remove repetated of same type entity
for i,_ in enumerate(TESTING_DATA):
    TESTING_DATA[i][1]['entities'] = ([next(b) for a, b in itertools.groupby( 
                         TESTING_DATA[i][1]['entities'], lambda y: y[2])]) 

In [0]:
LABEL = ['TITLE',
 'DESCRIPTION',
 'CLIENT',
 'ADDRESS',
 'SOCIAL_SECURITY_NO',
 'RELATIONSHIP_NO',
 'ACC_NO',
 'ACC_TYPE',
 'PHONE',
 'CITY',
 'EMAIL']

In [71]:
os.getcwd()

'/content'

In [72]:
# sets the model, output directory and training iterations 
model = None
output_dir=os.getcwd()
n_iter=100

# Checks to see if there is a current model or no model. In this case I will be starting with a blank model 

if model is not None:
    nlp1 = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp1 = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")



# create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp1.pipe_names:
    ner = nlp1.create_pipe('ner')
    nlp1.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp1.get_pipe('ner')

# add labels
for _, annotations in TRAINING_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

    # get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp1.pipe_names if pipe != 'ner']
with nlp1.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp1.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAINING_DATA)
        losses = {}
        for text, annotations in tqdm(TRAINING_DATA):
            nlp1.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

  0%|          | 0/54 [00:00<?, ?it/s]

Created blank 'en' model


  2%|▏         | 1/54 [00:00<00:06,  8.74it/s]

{'ner': 11946.789147525158}


  2%|▏         | 1/54 [00:00<00:08,  6.15it/s]

...

{'ner': 51.58389718387223}


  2%|▏         | 1/54 [00:00<00:09,  5.72it/s]

{'ner': 58.1277118224412}


100%|██████████| 54/54 [00:09<00:00,  5.60it/s]

{'ner': 53.531425698341295}


In [73]:
output_dir=os.getcwd()+'/model'
print(output_dir)

# save model to output directory
if output_dir is not None:
    output_dir = Path(output_dir)
if not output_dir.exists():
    output_dir.mkdir()
nlp1.to_disk(output_dir)
print("Saved model to", output_dir)

/content/model
Saved model to /content/model


In [74]:
# test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)

def predict_entities(text, nlp2):
    # test the trained model
    doc = nlp2(text)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return entities

Loading from /content/model


In [0]:
out = predict_entities(TEST_TEXTS[0], nlp2)

In [76]:
out

[('ACCOUNT TRANSFER FORM', 'TITLE'),
 ('404711495', 'SOCIAL_SECURITY_NO'),
 ('IRA', 'ACC_TYPE'),
 ('870214590', 'ACC_NO'),
 ('4989 Brighton Circle Road', 'ADDRESS'),
 ('(541) 754-3010', 'PHONE')]

In [77]:
out = predict_entities(TEST_TEXTS[1], nlp2)
out

[('ACCOUNT TRANSFER FORM', 'TITLE'),
 ('Mary H Craven', 'CLIENT'),
 ('545401875', 'SOCIAL_SECURITY_NO'),
 ('IRA', 'ACC_TYPE'),
 ('987012322', 'ACC_NO'),
 ('2397 Deans Lane', 'ADDRESS'),
 ('(541) 754-3055', 'PHONE')]

In [78]:
out = predict_entities(TEST_TEXTS[2], nlp2)
out

[('IRA', 'ACC_TYPE'),
 ('Account Application', 'TITLE'),
 ('Bobby J Cole', 'CLIENT'),
 ('Thomas L Dodds', 'CLIENT'),
 ('3977 Hillview Drive', 'ADDRESS'),
 ('(541) 754-1100', 'PHONE'),
 ('360-80-6429', 'SOCIAL_SECURITY_NO'),
 ('Bobby@gmail.com', 'EMAIL')]

In [79]:
out = predict_entities(TEST_TEXTS[3], nlp2)
out

[('ACCOUNT TRANSFER FORM', 'TITLE'),
 ('Joshua T Truitt', 'CLIENT'),
 ('545401855', 'SOCIAL_SECURITY_NO'),
 ('IRA', 'ACC_TYPE'),
 ('987012355', 'ACC_NO'),
 ('288 Park Ave 12th Floor', 'ADDRESS'),
 ('(541) 754-3055', 'PHONE')]

In [80]:
out = predict_entities(TEST_TEXTS[4], nlp2)
out

[('ACCOUNT TRANSFER FORM', 'TITLE'),
 ('Addie N McClellan', 'CLIENT'),
 ('545401811', 'SOCIAL_SECURITY_NO'),
 ('IRA', 'ACC_TYPE'),
 ('3835 Marigold Lane', 'ADDRESS'),
 ('(541) 754-3022', 'PHONE')]

In [81]:
out = predict_entities(TEST_TEXTS[5], nlp2)
out

[('IRA', 'ACC_TYPE'),
 ('Account Application', 'TITLE'),
 ('2465 Richison Drive', 'ADDRESS'),
 ('(541) 754-3011', 'PHONE'),
 ('360-80-6429', 'SOCIAL_SECURITY_NO'),
 ('Patricia@gmail.com', 'EMAIL')]

In [84]:
len(TEST_TEXTS)

6

In [85]:
TEST_TEXTS[5]

'CLEAR PRINT IRA Account Application charles SCHWAB Page 1 of 8 Investment Advisor ("IA") Information (This portion to be completed by IA.) IA Firm Name (Please print.): Patricia Kim IA Master Account Number: 7457 - 28 9 9 Service Team: IA Contact Information (if follow-up is required): Note: To transfer or distribute IRA assets to a beneficiary due to the death of the account holder, please use the inherited IRA Application. 1. Select IRA Type (Select only ONE type of account.) Traditional IRA. (Select only one.) Contributory IRA Rollover IRA Rollover from an employer retirement plan $ Name of Employer Sponsoring the Plan Approximate Total Value of Distribution Expected Distribution Date By signing this application, I elect that my IRA assets identified above be considered a rollover. This election is required required by the IRS to qualify my contribution as a rollover contribution. Roth IRA. (Select only one.) Roth Contributory Roth Conversion (from a Schwab Qualified Plan*) Current

In [0]:
# 

# Prediction - OCR then NER

In [86]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)

def predict_entities(text, nlp2):
    # test the trained model
    doc = nlp2(text)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return entities

Loading from /content/model


In [0]:
def predict_ocr_ner(path_): 
    print(path_)
    file_name = path_.split('/')[-1]
    print(file_name)
    if Path(file_name).suffix == '.pdf':
        extracted_text = detect_text_from_pdf(pdf_path = path_, num_of_pages = 2)
        return predict_entities(extracted_text, nlp2)
    elif Path(file_name).suffix == '.png' or 'jpg':
        extracted_text = detect_text_from_image(path_)
        return predict_entities(extracted_text, nlp2)
    else:
        pass

In [89]:
new_image = '/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Allan_A_Cook_.pdf'
result = predict_ocr_ner(new_image)
result

/content/drive/My Drive/DOC-NER/Ameritrade Institutional/Allan_A_Cook_.pdf
Allan_A_Cook_.pdf
Texts:
Texts:


[('ACCOUNT TRANSFER FORM', 'TITLE'),
 ('587102360', 'ACC_NO'),
 ('Allan A Cook', 'CLIENT'),
 ('404711495', 'SOCIAL_SECURITY_NO'),
 ('IRA', 'ACC_TYPE'),
 ('(541) 754-3012', 'PHONE')]

In [90]:
new_pdf = '/content/drive/My Drive/DOC-NER/Merrilllynch_CRA/Adrian Clemens.png'
result = predict_ocr_ner(new_pdf)
result

/content/drive/My Drive/DOC-NER/Merrilllynch_CRA/Adrian Clemens.png
Adrian Clemens.png
Texts:


[('Client Account Transfer Form', 'TITLE'),
 ('IRA', 'ACC_TYPE'),
 ('2 Seaport Ln', 'ADDRESS'),
 ('89101454', 'ACC_NO'),
 ('(541) 754-2545', 'PHONE'),
 ('Daniel C Paul', 'CLIENT'),
 ('Boston, MA,02210', 'CITY'),
 ('545401875', 'SOCIAL_SECURITY_NO'),
 ('335-0012', 'RELATIONSHIP_NO'),
 ('Adrian@gmail.com', 'EMAIL')]

In [0]:
import spacy 
from spacy.gold import GoldParse 
from spacy.scorer import Scorer
 
def evaluate(ner_model, examples):    
    scorer = Scorer()    
    for input_, annot in examples:        
        doc_gold_text = ner_model.make_doc(input_)        
        gold = GoldParse(doc_gold_text, entities=annot['entities'])        
        pred_value = ner_model(input_)        
        scorer.score(pred_value, gold)    
        return scorer.scores
 
output_dir = "model"
nlp2 = spacy.load(output_dir)
 
results = evaluate(nlp2, TESTING_DATA)

In [95]:
results

{'ents_f': 92.3076923076923,
 'ents_p': 100.0,
 'ents_per_type': {'ACC_NO': {'f': 100.0, 'p': 100.0, 'r': 100.0},
  'ACC_TYPE': {'f': 100.0, 'p': 100.0, 'r': 100.0},
  'ADDRESS': {'f': 100.0, 'p': 100.0, 'r': 100.0},
  'CLIENT': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'PHONE': {'f': 100.0, 'p': 100.0, 'r': 100.0},
  'SOCIAL_SECURITY_NO': {'f': 100.0, 'p': 100.0, 'r': 100.0},
  'TITLE': {'f': 100.0, 'p': 100.0, 'r': 100.0}},
 'ents_r': 85.71428571428571,
 'las': 0.0,
 'las_per_type': {'': {'f': 0.0, 'p': 0.0, 'r': 0.0}},
 'tags_acc': 0.0,
 'textcat_score': 0.0,
 'textcats_per_cat': {},
 'token_acc': 100.0,
 'uas': 0.0}